In [1]:
import numpy as np 
import tensorflow as tf
import keras
import matplotlib
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import InputLayer, Dense, Flatten, Dropout, Conv2D, MaxPooling2D
from tensorflow.keras import utils as np_utils
import numpy as np
from sklearn.model_selection import StratifiedKFold

In [2]:
(x,y), (x_test, y_test) = mnist.load_data()
x=x.reshape(x.shape[0],28,28,1)
x=x.astype("float32")
x/=255
y = np_utils.to_categorical(y,10)

In [3]:
seed = 5
np.random.seed(seed)

In [4]:
kfold = StratifiedKFold(n_splits = 5, shuffle = True, random_state = seed) #como cv, divisao de partes, dados embaralhados para nao acostumar a rede neural

In [5]:
from sklearn.model_selection import KFold
import numpy as np
from keras.models import Sequential
from keras.layers import InputLayer, Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam

# Resultados da precisão de cada fold
resultados = []

# Definindo o KFold (n_splits=5 para 5 divisões)
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Loop de cross-validation
for indice_train, indice_test in kfold.split(x, y):  # Divisão dos dados
    print(f"Índices Train: {indice_train[:5]}...  |  Índices Test: {indice_test[:5]}...")  # Mostrando os primeiros indices para conferirmos
    classificador = Sequential()
    
    # Camada de entrada
    classificador.add(InputLayer(shape=(28, 28, 1)))  # Alterado para `shape` em vez de `input_shape`
    
    # Primeira camada convolucional
    classificador.add(Conv2D(32, (3, 3), activation='relu', padding='same'))
    classificador.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Segunda camada convolucional
    classificador.add(Conv2D(64, (3, 3), activation='relu', padding='same'))
    classificador.add(MaxPooling2D(pool_size=(2, 2)))
    
    # Flatten a saída da última camada convolucional
    classificador.add(Flatten())
    
    # Camada densa com Dropout
    classificador.add(Dense(units=128, activation='relu'))
    classificador.add(Dropout(0.3))  # Dropout para prevenir overfitting
    
    # Camada de saída
    classificador.add(Dense(units=10, activation='softmax'))
    
    # Compilação do modelo
    classificador.compile(loss='categorical_crossentropy', 
                          optimizer=Adam(learning_rate=0.001), 
                          metrics=['accuracy'])
    
    # Treinando o modelo
    classificador.fit(x[indice_train], y[indice_train], batch_size=128, epochs=5, verbose=1)
    
    # Avaliando o modelo no conjunto de teste
    precisao = classificador.evaluate(x[indice_test], y[indice_test], verbose=0)
    
    # Imprimindo a precisão para o fold atual
    print(f"Precisão no Fold: {precisao[1]:.4f}")
    
    # Adicionando a precisão ao resultado
    resultados.append(precisao[1])

# Calculando a média da precisão dos folds
media_precisao = np.mean(resultados)
print(f"Precisão média nos folds: {media_precisao:.4f}")


Índices Train: [0 1 2 3 5]...  |  Índices Test: [ 4  6  7 32 34]...
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.8127 - loss: 0.6035
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 28ms/step - accuracy: 0.9751 - loss: 0.0851
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 29ms/step - accuracy: 0.9831 - loss: 0.0557
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 11s 30ms/step - accuracy: 0.9859 - loss: 0.0438
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 14s 38ms/step - accuracy: 0.9888 - loss: 0.0359
Precisão no Fold: 0.9895
Índices Train: [2 3 4 5 6]...  |  Índices Test: [ 0  1 11 13 17]...
Epoch 1/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 21s 55ms/step - accuracy: 0.8090 - loss: 0.6148
Epoch 2/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 25s 67ms/step - accuracy: 0.9757 - loss: 0.0798
Epoch 3/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 37s 55ms/step - accuracy: 0.9828 - loss: 0.0541
Epoch 4/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 19s 50ms/step - accuracy: 0.9868 - loss: 0.0406
Epoch 5/5
375/375 ━━━━━━━━━━━━━━━━━━━━ 20s 53ms/step - a

In [10]:
np.array(resultados).mean()

0.9883000016212463

In [11]:
np.array(resultados).std() #desvio padrao baixo = confiavel

0.0009495611081096058

In [12]:
resultados

[0.9894999861717224,
 0.9890833497047424,
 0.9883333444595337,
 0.9877499938011169,
 0.9868333339691162]